In [1]:
import os
import sys
from time import time
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import networkx as nx
import random

from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks


from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')
from PatchyConverter import PatchyConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier


import matplotlib.pyplot as plt
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load data

In [2]:
GAMMA_PATH = os.environ.get('GAMMA_DATA_ROOT')

In [54]:
dataset_name = 'Samples/karate'
path= os.path.join(GAMMA_PATH,dataset_name)

In [104]:
def check_header(filename):
    with open(filename) as f:
        first = f.read(1)
    return first not in '.-0123456789'

def rename_cols(df,list_new_names):
    renaming_dict = {i:j for i,j in zip(df.columns.tolist(),list_new_names)}
    df.rename(columns=renaming_dict,inplace=True)

def import_edgelist(path):
    if check_header(path) == True:
        adj_list = pd.read_csv(path,header=0)
        rename_cols(adj_list,['from','to'])
    else:
        adj_list = pd.read_csv(path,header=None)
        adj_list.columns = ['from','to']
    return adj_list

def import_node_labels(path):
    if check_header(path) == True:
        df_node_label = pd.read_csv(path,header=0)
        rename_cols(df_node_label,['node','label'])
    else:
        df_node_label = pd.read_csv(path,header=None)
        df_node_label.columns = ['node','label']
    return adj_list

In [105]:
path_nodes = os.path.join(path,'nodes.csv')
path_edges = os.path.join(path,'edge_list.csv')
path_node_labels = os.path.join(path,'node_labels.csv')
#path_labelled_nodes = os.path.join(path,'labelled_nodes.csv')

#path_groups= os.path.join(path,'groups.csv')
#path_node_labels = os.path.join(path,'labelled_nodes.csv')


In [106]:
df_edges = import_edgelist(path_edges)

,from,to
0,1,2
1,1,3
2,1,4
3,1,5
4,1,6
5,1,7
6,1,8
7,1,9
8,1,11
9,1,12


In [64]:
str.filter(,'from')

AttributeError: type object 'str' has no attribute 'filter'

In [50]:
# Importing nodes:
df_nodes = pd.read_csv(path_nodes,header= None)
df_nodes.columns = ['node']

# Importing edges:
df_edges = import_edgelist(path_edges)

# Importing labels:
df_node_labels = pd.read_csv(path_node_labels,header= None)
df_node_labels.columns = ['node','label']
# def import_labelled_data():
#     df_node_labels.columns = ['node','label']
#     df_node_labels.columns = ['label']
#     df_node_labels['label'] = df_node_labels['label'].astype(int)
#     df_node_labels['node'] = df_node_labels.index
#     df_node_labels= df_node_labels.loc[:,['node','label']]

In [43]:
pd.read_csv(path_labelled_nodes, header=None)

,0
0,1.0
1,3.0
2,5.0
3,8.0
4,13.0
5,15.0
6,23.0
7,24.0
8,25.0
9,34.0


In [43]:
pd.read_csv(path,sep =' ',header=None).

(1005, 2)

In [ ]:
dropbox_loader = DropboxLoader(dataset_name)
graph_labels = dropbox_loader.get_graph_label()
graph_labels = np.array(graph_labels.graph_label)

In [ ]:
# Capsule Architecture Parameters:
capsule_params= CapsuleParameters()

# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'

capsule_params.add_params(conv_layer_params,conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2,2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params,caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params,digit_layer_name )

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 256 #250 #512
decoder_params['second_dense'] = 512
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params,decoder_layer)


In [ ]:
parameter_set = CapsuleTrainingParameters()
parameter_set.add_fold(1)
parameter_set.epochs= 5

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(graph_tensor,graph_labels,test_size=0.10,random_state=j)
#data = ((x_train, y_train), (x_test, y_test))
#input_shape = x_train.shape[1:]

In [ ]:
acc_list = []
for j in range(10):
    x_train, x_test, y_train, y_test = train_test_split(graph_tensor,graph_labels,test_size=0.10,random_state=j)
    #data = ((x_train, y_train), (x_test, y_test))
    #input_shape = x_train.shape[1:]
    X = x_train.reshape(x_train.shape[0],-1)
    y = y_train.reshape(y_train.shape[0],-1)
    clf = svm.SVC()
    clf.fit(X, y)  
    X_test_svm = x_test.reshape(x_test.shape[0],-1)
    y_test_svm = y_test.reshape(y_test.shape[0],-1)
    y_pred = clf.predict(X_test_svm)
    acc = accuracy_score(y_pred,y_test_svm)
    acc_list.append(acc)
    print(acc)
print('mean acc : {}'.format(np.mean(acc_list)))
print('std acc : {}'.format(np.std(acc_list)))
    

In [ ]:
# patchy_classifier = GraphClassifier(input_shape,6)
# patchy_classifier.build_the_graph(capsule_params)
# patchy_classifier.train(data, parameter_set)
#patchy_classifier.train_model.summary()

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score